In [4]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import geopandas as gpd
import glob
from pyproj import CRS
from shapely.geometry import Point
from datetime import timedelta
import movingpandas as mpd
import matplotlib.pyplot as plt
import uuid
import math

In [5]:
''' 
When reading all users, don't forget to check for the labels.txt file
'''
def read_users(folder):
    subfolders = os.listdir(folder)
    dfs = []
    
    # df in which each row is a single trajectory
    transport_df = pd.DataFrame(columns=['Start Time','End Time','Transportation Mode'])
    
    # df of all GPS points recorded
    gps_points_df = pd.DataFrame(columns=['Identifier', 'Timestamp', 'Latitude', 'Longitude', 'Altitude', 'Label'])
    
    for i, sf in enumerate(subfolders):
        user_folder = os.path.join(folder,sf)
        labels_file = os.path.join(user_folder, 'labels.txt')
        if os.path.exists(labels_file):
            print('Processing user %s ---------------------------' % (sf))
            
            # single user df in which each row is a single trajectory
            u_transport_df = pd.read_csv(labels_file, sep="\t")
            u_transport_df['Identifier'] = 0
            
            # single user df of all GPS points recorded
            u_gps_points_df = pd.DataFrame(columns=['Identifier', 'Timestamp', 'Latitude', 'Longitude', 'Altitude', 'Label'])
                
            plt_files = os.path.join(user_folder, 'Trajectory')
            
            # searching for the corresponding trajectory for present .plt file based on timestamp
            for index, row in u_transport_df.iterrows():
                start = datetime.strptime(row['Start Time'],'%Y/%m/%d %H:%M:%S')
                end = datetime.strptime(row['End Time'],'%Y/%m/%d %H:%M:%S')

                #creating a random identifier for each trajectory so we can link them to the GPS points
                identifier = uuid.uuid1()
                u_transport_df.loc[index, 'Identifier'] = identifier

                # iterating through .plt files in Trajectory folder
                for filename in os.listdir(plt_files):
                    dt_object = datetime.strptime(filename[:-4], '%Y%m%d%H%M%S')
                
                    if dt_object >= start and dt_object <= end:
                        # turns .plt file into dataframe so it can be added to u_gps_points_df
                        file_df = pd.read_csv(os.path.join(plt_files, filename), skiprows=6, header=None,
                                 parse_dates=[[5, 6]], infer_datetime_format=True)
                        file_df.rename(inplace=True, columns={'5_6': 'Timestamp', 0: 'Latitude', 
                                                              1: 'Longitude', 3: 'Altitude'})
                        file_df.drop(inplace=True, columns=[2, 4])

                        # receives identifier to indicate which trajectory it belongs to
                        file_df['Identifier'] = identifier

                        file_df['Label'] = u_transport_df.loc[index, 'Transportation Mode']

                        u_gps_points_df = pd.concat((u_gps_points_df,file_df), axis=0)
                        
            transport_df = pd.concat((transport_df,u_transport_df), axis=0)
            gps_points_df = pd.concat((gps_points_df,u_gps_points_df), axis=0)
    
    return transport_df,gps_points_df

start = datetime.now()
transport_df,gps_points_df = read_users('../Data')

end = datetime.now()
print('Tempo gasto:',end-start)

Processing user 010 ---------------------------
Processing user 020 ---------------------------
Processing user 021 ---------------------------
Processing user 052 ---------------------------
Processing user 053 ---------------------------
Processing user 056 ---------------------------
Processing user 058 ---------------------------
Processing user 059 ---------------------------
Processing user 060 ---------------------------
Processing user 062 ---------------------------
Processing user 064 ---------------------------
Processing user 065 ---------------------------
Processing user 067 ---------------------------
Processing user 068 ---------------------------
Processing user 069 ---------------------------
Processing user 073 ---------------------------
Processing user 075 ---------------------------
Processing user 076 ---------------------------
Processing user 078 ---------------------------
Processing user 080 ---------------------------
Processing user 081 --------------------

In [6]:
transport_df

,Start Time,End Time,Transportation Mode,Identifier
0,2007/06/26 11:32:29,2007/06/26 11:40:29,bus,0efa4a9c-f70a-11ec-abef-74867afcda99
1,2008/03/28 14:52:54,2008/03/28 15:59:59,train,0efae6c2-f70a-11ec-aeb2-74867afcda99
2,2008/03/28 16:00:00,2008/03/28 22:02:00,train,0efb34d5-f70a-11ec-867a-74867afcda99
3,2008/03/29 01:27:50,2008/03/29 15:59:59,train,0f0aaedf-f70a-11ec-98e9-74867afcda99
4,2008/03/29 16:00:00,2008/03/30 15:59:59,train,0f0bc024-f70a-11ec-a0ed-74867afcda99
...,...,...,...,...
314,2008/11/17 06:59:58,2008/11/17 07:06:16,bus,c4562e20-f70a-11ec-be4d-74867afcda99
315,2008/11/17 07:06:16,2008/11/17 07:14:32,walk,c456a33c-f70a-11ec-91d8-74867afcda99
316,2008/11/29 01:58:05,2008/11/29 02:01:39,bus,c4571859-f70a-11ec-aeba-74867afcda99
317,2008/11/29 02:01:39,2008/11/29 02:07:57,walk,c45cbc88-f70a-11ec-b9f9-74867afcda99


In [9]:
gps_points_df

,Identifier,Timestamp,Latitude,Longitude,Altitude,Label
0,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:00:01,39.50293,116.714948,-777,train
1,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:01:00,39.497045,116.726137,-777,train
2,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:01:59,39.489695,116.740047,-777,train
3,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:02:59,39.481438,116.755648,-777,train
4,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:03:58,39.472748,116.770972,-777,train
...,...,...,...,...,...,...
4704,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:52,40.007802,116.319362,84,bus
4705,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:54,40.00778,116.31936,88,bus
4706,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:56,40.007756,116.319362,92,bus
4707,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:58,40.00774,116.319361,97,bus


In [10]:
from sklearn import preprocessing

gps_points_df_scaled = gps_points_df.copy()

values = gps_points_df_scaled['Latitude'].values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler((-90,90))
scaled = min_max_scaler.fit_transform(values)
gps_points_df_scaled['Latitude'] = pd.DataFrame(scaled)

gps_points_df_scaled['geometry'] = [Point(lon, lat) for lon, lat in 
                                 zip(gps_points_df_scaled['Longitude'].to_list(), gps_points_df_scaled['Latitude'].to_list())]
gps_points_df_scaled

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Identifier,Timestamp,Latitude,Longitude,Altitude,Label,geometry
0,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:00:01,-79.983301,116.714948,-777,train,POINT (116.714948 -79.98330116950432)
1,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:01:00,-79.986075,116.726137,-777,train,POINT (116.726137 -79.98607481050473)
2,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:01:59,-79.989539,116.740047,-777,train,POINT (116.740047 -79.98953891608724)
3,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:02:59,-79.993430,116.755648,-777,train,POINT (116.755648 -79.9934304970117)
4,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:03:58,-79.997526,116.770972,-777,train,POINT (116.770972 -79.99752615381605)
...,...,...,...,...,...,...,...
4704,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:52,-79.818221,116.319362,84,bus,POINT (116.319362 -79.81822122102406)
4705,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:54,-79.818241,116.31936,88,bus,POINT (116.31936 -79.81824148721998)
4706,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:56,-79.818250,116.319362,92,bus,POINT (116.319362 -79.81825044205074)
4707,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:15:58,-79.818193,116.319361,97,bus,POINT (116.319361 -79.81819294261113)


In [11]:
start = datetime.now()

# Creating a Geodataframe. Be aware it is CRS 4326 WGS84
# A GeoDataFrame object is a pandas.DataFrame that has a column with geometry
# It has one GeoSeries column that holds a special status. 
# This GeoSeries is referred to as the GeoDataFrame’s “geometry”. 
# A GeoSeries is essentially a vector where each entry in the vector is a set 
# of shapes corresponding to one observation.
geodata = gpd.GeoDataFrame(gps_points_df_scaled, crs = CRS.from_epsg('4326'))
geodata = geodata.set_index('Timestamp')

geo = datetime.now()
print('Tempo gasto criando geodata:',geo-start)

# Create a Trajectory Collection with Movingpandas
# data(list[Trajectory] or GeoDataFrame or DataFrame)–List of Trajectory 
#     objects or a GeoDataFrame with trajectory IDs, point geometry column 
#     and timestamp index
# traj_id_col(string)–Name of the GeoDataFrame column containing trajectory IDs
traj_collection = mpd.TrajectoryCollection(geodata, 'Identifier')

traj = datetime.now()

print('Tempo gasto criando TrajectoryCollection:',traj-geo)

start = datetime.now()

# Detects stops in a trajectory. A stop is detected if the movement stays 
# within an area of specified size for at least the specified duration.
# Define parameters in Hours and Search radius in meters
Hours = .01
SearchRadio = 500
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), 
                                                                      max_diameter=SearchRadio)

end = datetime.now()
print('Tempo gasto detectando stop points:',end-start)

stops

Tempo gasto criando geodata: 0:00:55.511641
Tempo gasto criando TrajectoryCollection: 0:22:39.756680


C:\ProgramData\Miniconda3\lib\site-packages\movingpandas\geometry_utils.py:150: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(geom) == 2:
C:\ProgramData\Miniconda3\lib\site-packages\movingpandas\geometry_utils.py:151: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return _measure_distance(geom[0], geom[1], spherical)


Tempo gasto detectando stop points: 2:01:45.801712


TrajectoryCollection with 47411 trajectories

In [12]:
start = datetime.now()

# Create a new Geodataframe and define geometry column
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')

# Add the ID of each stop track and define it as index
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start = stops_start.set_index('stop_id')

# Iteration over the Stop Trajectories
for stoptrack in stops.trajectories:

    # add stop duration in hours
    stops_start.at[stoptrack.id,'duration_s'] = stoptrack.get_duration().total_seconds()

    # add length
    stops_start.at[stoptrack.id, 'length_m'] = stoptrack.get_length()

    # add traj id
    stops_start.at[stoptrack.id, 'identifier'] = stoptrack.id.split('_')[0]

    # add datetime
    stops_start.at[stoptrack.id, 'datetime']  = pd.to_datetime(stoptrack.id.split('_')[1]).tz_localize(None)

    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()
        
# Reset indexes
stops_start = stops_start.reset_index(drop=True)
geodata = geodata.reset_index(drop=True)

end = datetime.now()
print('Tempo gasto criando stops df:',end-start)

stops_start

Tempo gasto criando stops df: 0:10:48.841367


,geometry,duration_s,length_m,identifier,datetime
0,POINT (116.71495 -79.98330),59.0,378.346177,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:00:01
1,POINT (117.13302 -80.09865),59.0,359.821595,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:25:43
2,POINT (117.14632 -80.10710),59.0,403.652976,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:28:41
3,POINT (117.16538 -80.11921),178.0,496.751065,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:31:38
4,POINT (117.17803 -80.12692),59.0,371.201060,0efb34d5-f70a-11ec-867a-74867afcda99,2008-03-28 16:36:34
...,...,...,...,...,...
47406,POINT (116.32467 -79.87805),76.0,431.416024,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 07:12:12
47407,POINT (116.32426 -79.99414),540.0,769.930472,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 07:14:40
47408,POINT (116.31937 -79.99638),2858.0,1397.116558,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 07:23:44
47409,POINT (116.31940 -79.99556),56.0,484.230400,c4571859-f70a-11ec-aeba-74867afcda99,2008-11-29 08:11:26


In [52]:
from math import log

start = datetime.now()

point_density_table = []

######### compare points in stops_start and geodata. 
######### try plotting each batch of stops_start points by traj ID
ids = stops_start['identifier'].unique()

for i in ids:
    fig = plt.figure()
    # filtering stop points by id and plotting them
    filtered_stops = stops_start[stops_start['identifier']==i]
    lat_stops = [p.y for p in filtered_stops['geometry']]
    lon_stops = [p.x for p in filtered_stops['geometry']]
    
    plt.scatter(lon_stops,lat_stops)
    plt.plot(lon_stops,lat_stops)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    
    label = transport_df[transport_df['Identifier']==uuid.UUID(i)]['Transportation Mode'].values[0]
    title = i+'_'+label
    plt.title(title)
    
    plt.axis('off')
    plt.savefig('traj_images/'+title+'.png')
    
    plt.close(fig)
    
    # getting info to calculate point density
    num_stops = len(filtered_stops['length_m'])
    dist = sum(filtered_stops['length_m'])
    duration_avg = np.mean(filtered_stops['duration_s'])
    info = [label, num_stops, duration_avg, dist, num_stops/dist]
    point_density_table.append(info)
    
end = datetime.now()
print('Tempo gasto gerando imagens:',end-start)

Tempo gasto gerando imagens: 0:03:04.359489


In [53]:
point_density_table = pd.DataFrame.from_records(point_density_table, columns = ['Label','Num Stops', 'Avg Duration', 'Dist', 'Ratio'])
point_density_table

,Label,Num Stops,Avg Duration,Dist,Ratio
0,train,196,128.311224,76266.168136,0.002570
1,train,99,131.121212,55975.219213,0.001769
2,train,113,148.168142,59897.074871,0.001887
3,taxi,140,151.921429,81147.677187,0.001725
4,walk,15,77.666667,10467.554477,0.001433
...,...,...,...,...,...
3384,walk,2,54.000000,948.483206,0.002109
3385,bike,15,374.266667,9764.682644,0.001536
3386,bike,10,77.400000,6618.042374,0.001511
3387,bus,24,410.875000,15076.571432,0.001592


In [56]:
from sklearn.model_selection import train_test_split

X = point_density_table[['Num Stops', 'Dist', 'Ratio']]
y = point_density_table['Label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

start = datetime.now()

depths = [2,3,5]
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average='micro')
    print(score)
end = datetime.now()
print('Tempo gasto:',end-start)

0.3702064896755162
0.37315634218289084
0.3923303834808259
Tempo gasto: 0:00:00.034978


In [58]:
# '''
# Don't forget about the distribution of labels, because it is probably unbalanced
# Maybe there is a relationship between the distance covered and the number of stop points? Like density?
# Because a faster mode will have a ratio of distance/stops smaller than a slower mode?
# '''

# import plotly.express as px

# fig = px.scatter_matrix(point_density_table,
#     dimensions=["Num Stops", "Dist", "Ratio"],
#     color="Label")
# fig.show()

In [ ]:
# import matplotlib.pyplot as plt

# def draw_motif(transport_df,gps_points_df):
#     ids = gps_points_df['identifier'].unique()
#     for i in ids[:5]:
#         single_mode_df = gps_points_df[gps_points_df['identifier']==i]
#         plt.scatter(x=single_mode_df['lon'], y=single_mode_df['lat'])
#         plt.xlabel('Longitude')
#         plt.ylabel('Latitude')
#         plt.title(transport_df.at[i,'Transportation Mode'])
#         plt.show()
        
# draw_motif(transport_df,gps_points_df)

#     stops = stops_start['geometry'].tolist()
#     print('stops #######################\n',stops)
#     xs = [point.x for point in stops]
#     ys = [point.y for point in stops]
#     plt.scatter(xs, ys)
#     plt.plot(xs, ys)
#     plt.title('Transport Mode:'+transport_df.at[i,'Transportation Mode'])
#     plt.show()